## Preparando os dados para submeter ao Modelo de Machine Lernaning

#### Aqui veremos o básico de como preparamos nossos dados para submetê-lo ao modelo de Machine Learning. Lembrando que antes dessa preparação, primeiro precisamos realizar todo um processo de transformação, limpeza da base, tratamento de dados faltantes e inconsistentes, além de algumas análises exploratórias para que possamos nos familiarizar com os dados e através deles, fazermos perguntas e entender quais respostas poderemos obter com eles.

In [1]:
# Começaremos importando o pandas para lermos a base de dados. Aqui usaremos uma base simples e conhecida do Censo.
import pandas as pd

In [2]:
base = pd.read_csv('census.csv')

In [5]:
base.shape

(32561, 15)

In [3]:
base.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### O primeiro passo é transformar nossas variáveis categóricas em numéricas, pois alguns modelos não aceitam variáveis categóricas (Nominais, Ordinais). Mas não podemos transformar em qualquer número, para isso precisamos codificar essas variáveis de forma a não atribuir um peso maior para alguma delas sem querer, atrubuindo por exemplo uma escala (1,2,3,4,5), etc. Ainda mais se nossa variável não tem uma relação de ordenação ou prioridade entre seus valores.

#### A melhor prática para essa transformação é criar variáveis Dummys (variáveis binárias (0 ou 1) criadas para representar uma variável com duas ou mais categorias). Podemos fazer isso de forma bastante simples uando o método OneHotEncoder(), do ScikitLearn. Vejamos abaixo os passos:

In [26]:
base.columns

Index(['age', 'workclass', 'final-weight', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loos', 'hour-per-week', 'native-country',
       'income'],
      dtype='object')

In [25]:
# Variável nominal
base.workclass.value_counts()

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64

In [29]:
# variável ordinal
base['education'].value_counts()

 HS-grad         10501
 Some-college     7291
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctorate         413
 5th-6th           333
 1st-4th           168
 Preschool          51
Name: education, dtype: int64

#### Vamos primeiro dividir nossa base entre as variáveis previsoras e a classe ( a qual queremos prever).

In [6]:
# As variáveis previsoras vão da coluna de índice 0 até 13, para incluirmos a coluna 13, precisamos passar nossa fatia até 14.
previsores = base.iloc[:, 0:14].values

In [8]:
# Observamos que nossa base "previsores", possui nossas 14 colunas de varoáveis previsoras (índices de 0 a 13)
previsores.shape

(32561, 14)

In [9]:
# Como a nossa coluna que contém a variável classe é a de índice 14, passamos somente ela em nosso fatiamento:
classe = base.iloc[:, 14].values

In [10]:
# Dando uma espiada em nossa nova base "classe", podemos ver que possui somente valores da coluna 14
classe

array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' <=50K', ' >50K'],
      dtype=object)

#### Agora sim, iremos importar os pacotes para transformarmos nossas variáveis em dummys

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [13]:
from sklearn.compose import ColumnTransformer

In [14]:
##labelencoder_previsores = LabelEncoder()

### As colunas categóricas que queremos transformar, são as de índices 1,3,5,6,7,8,9,13.
#### Passaremos uma lista com essas variáveis no nosso código para realizar a transformação delas em dummys. O número de dummys que serão criadas, depende da quantidade de categorias que temos em cada uma dessas variáveis. 

In [15]:
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [1,3,5,6,7,8,9,13])],remainder='passthrough')

In [17]:
previsores = onehotencoder.fit_transform(previsores).toarray()

In [22]:
# Podemos ver que as nossas 14 variáveis se tornaram 108
previsores.shape

(32561, 108)

In [20]:
# vamos codificar nossa classe também para que os valores sejam normalizados para que contenham apenas valores entre 0 e n_classes -1
labelencoder_classe = LabelEncoder()
classe = labelencoder_classe.fit_transform(classe)

In [21]:
# Como ficou nossa classe após aplicarmos a codificação
classe

array([0, 0, 0, ..., 0, 0, 1])

#### Por fim, vamos transformar todas as variáveis na mesma ordem de grandeza. Para isso iremos utilizar a técnica de padronização

In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [24]:
# Agora temos nossas variáveis previsoras, transformadas e padronizadas. Passaremos ao próximo passo de dividir nossa bae
previsores

array([[-0.2444502 , -0.17429511, -0.26209736, ...,  0.1484529 ,
        -0.21665953, -0.03542945],
       [-0.2444502 , -0.17429511, -0.26209736, ..., -0.14592048,
        -0.21665953, -2.22215312],
       [-0.2444502 , -0.17429511, -0.26209736, ..., -0.14592048,
        -0.21665953, -0.03542945],
       ...,
       [-0.2444502 , -0.17429511, -0.26209736, ..., -0.14592048,
        -0.21665953, -0.03542945],
       [-0.2444502 , -0.17429511, -0.26209736, ..., -0.14592048,
        -0.21665953, -1.65522476],
       [-0.2444502 , -0.17429511, -0.26209736, ...,  1.88842434,
        -0.21665953, -0.03542945]])

#### Divisão das bases de dados (Treinamento e Teste)

In [31]:
## Vamos importar o pacote para realizarmos a divisão
from sklearn.model_selection import train_test_split

In [36]:
# Aqui criaremos base de Treino e Teste para cada uma de nossas bases (previsoras e classe)
# Como temos uma quentidade razoável de observações (linhas), iremos pegar 15% para nossa base de teste, o restante ficará
# com a base de treino.
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.15, random_state=0)

In [37]:
previsores_teste.shape

(4885, 108)

In [38]:
previsores_treinamento.shape

(27676, 108)

In [39]:
classe_teste.shape

(4885,)

In [40]:
classe_treinamento.shape

(27676,)

**FIM!!!!**